# Syft Keras के साथ निजी भविष्यवाणियाँ  (Private Predictions with Syft Keras)

# चरण 1: सार्वजनिक प्रशिक्षण

इस ट्यूटोरियल में आपका स्वागत है! निम्नलिखित पुस्तिकाओं में आप जानेंगे कि निजी भविष्यवाणियाँ कैसे प्रदान करें। निजी भविष्यवाणियों द्वारा, हमारा मतलब है कि डेटा पूरी प्रक्रिया में लगातार एन्क्रिप्टेड है। किसी भी बिंदु पर उपयोगकर्ता कच्चे डेटा को साझा नहीं कर रहा है, केवल एन्क्रिप्टेड (यानी गुप्त साझा) डेटा। इन निजी भविष्यवाणियों को प्रदान करने के लिए, `Syft Keras` हुड के तहत [TF Encrypted](https://github.com/tf-encrypted/tf-encrypted) नामक एक पुस्तकालय का उपयोग करता है। TF एन्क्रिप्टेड अत्याधुनिक क्रिप्टोग्राफिक और मशीन लर्निंग तकनीकों को जोड़ती है, लेकिन आपको इसके बारे में चिंता करने की ज़रूरत नहीं है और अपने मशीन लर्निंग एप्लिकेशन पर ध्यान केंद्रित कर सकते हैं।

आप केवल तीन चरणों के साथ निजी भविष्यवाणियों की सेवा शुरू कर सकते हैं:
- **चरण 1**: सामान्य केरस के साथ अपने मॉडल को प्रशिक्षित करें।
- **चरण 2**: अपने मशीन लर्निंग मॉडल (सर्वर) को सुरक्षित करे और सेवा दें।
- **चरण 3**: निजी पूर्वानुमान (ग्राहक) प्राप्त करने के लिए सुरक्षित मॉडल को क्वेरी करें।

ठीक है, चलो इन तीन चरणों के माध्यम से चलते हैं ताकि आप उपयोगकर्ता की गोपनीयता या मॉडल सुरक्षा का त्याग किए बिना प्रभावशाली मशीन सीखने की सेवाओं को तैनात कर सकें।

लेखक:
- Jason Mancuso - Twitter: [@jvmancuso](https://twitter.com/jvmancuso)
- Yann Dupis - Twitter: [@YannDupis](https://twitter.com/YannDupis)
- Morten Dahl - Twitter: [@mortendahlcs](https://github.com/mortendahlcs)

अनुवादक:
- Yugandhar Tripathi - Twitter: [@Yugandhar_19](https://twitter.com/Yugandhar_19) · Github: [@Yugandhartripathi](https://github.com/Yugandhartripathi)

इनकी ओर से:
- Dropout Labs - Twitter: [@dropoutlabs](https://twitter.com/dropoutlabs)
- TF Encrypted - Twitter: [@tf_encrypted](https://twitter.com/tf_encrypted)

## अपने मॉडल को Keras के साथ प्रशिक्षित करें

अपनी परियोजनाओं के लिए गोपनीयता-संरक्षण मशीन सीखने की तकनीक का उपयोग करने के लिए आपको एक नई मशीन सीखने की रूपरेखा सीखने की आवश्यकता नहीं होनी चाहिए। यदि आपके पास बुनियादी [Keras](https://keras.io/) ज्ञान है, तो आप इन तकनीकों का उपयोग Syft Keras के साथ शुरू कर सकते हैं। यदि आपने पहले कभी Keras का उपयोग नहीं किया है, तो आप [Keras प्रलेखन](https://keras.io) के माध्यम से इसके बारे में थोड़ा और जान सकते हैं।

निजी भविष्यवाणियों की सेवा करने से पहले, अपने मॉडल को सामान्य केरस के साथ प्रशिक्षित करना है। एक उदाहरण के रूप में, हम हस्तलिखित अंकों को वर्गीकृत करने के लिए एक मॉडल को प्रशिक्षित करेंगे। इस मॉडल को प्रशिक्षित करने के लिए हम कैनोनिकल [MNIST डेटासेट](http://yann.lecun.com/exdb/mnist/) का उपयोग करेंगे।

हमने [Keras उदाहरण भंडार](https://github.com/keras-team/keras/blob/master/examples/mnist_cnn.py) से उधार लिया। अपने वर्गीकरण मॉडल को प्रशिक्षित करने के लिए, आप बस नीचे दिए गए सेल को चलाते हैं।

In [ ]:
from __future__ import print_function
import tensorflow.keras as keras
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, AveragePooling2D
from tensorflow.keras.layers import Activation

batch_size = 128
num_classes = 10
epochs = 2

# input image dimensions
img_rows, img_cols = 28, 28

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

model = Sequential()

model.add(Conv2D(10, (3, 3), input_shape=input_shape))
model.add(AveragePooling2D((2, 2)))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(AveragePooling2D((2, 2)))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(AveragePooling2D((2, 2)))
model.add(Activation('relu'))
model.add(Flatten())
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adam(),
              metrics=['accuracy'])

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

## भविष्य की निजी भविष्यवाणी के लिए अपने मॉडल के वजन को सहेजें

बहुत बढ़िया! आपका मॉडल प्रशिक्षित है। चलो `model.save()` के साथ मॉडल वज़न सहेजें। अगली नोटबुक में, हम निजी भविष्यवाणियों की सेवा शुरू करने के लिए Syft Keras में इन भारों को लोड करेंगे।

In [ ]:
model.save('short-conv-mnist.h5')